In [1]:
from scripts.functions import *

In [2]:
from parameters import *

In [41]:
import glob
import multiprocessing
from datetime import datetime
import itables.interactive

In [80]:
def get_existing_orders(client, pages=None):
    # Search all the requested orders per page
    # Fixed api.models NEXT_KEY parameter from "_next" to "next"

    ordered_orders = client.get_orders()
    ordered_orders.NEXT_KEY = "next"
    order_pages=[]

    # We can limit the search to certain number of pages
    # if we leave as none, will search over all of them
    limit_to_x_pages = pages
    for page in ordered_orders.iter(limit_to_x_pages):
        page.NEXT_KEY = "next"
        order_pages.append(page.get())

    current_server_orders = [order for page in order_pages for order in page['orders']]
    
    return current_server_orders



def get_orders_status(client, pages=None):
    current_server_orders = get_existing_orders(client, pages)
    progress_df = pd.DataFrame([(f['created_on'], 
                                 f['last_message'], 
                                 f['last_modified'], 
                                 f['id'], 
                                 f['name'], 
                                 f['state'], ) for f in current_server_orders])
    progress_df.columns =['created_on', 'last_message', 'last_modified', 'id', 'name', 'state',]
    progress_df.sort_values(by=['created_on'])
    
    return progress_df

to use your own api keys and parameters, copy paste the `parameters.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own keys. only the .dist will be pushed to the dist git rep. 

# 1. Search items
### Get the samples dataframe

From a geojson plots file, create a geo pandas dataframe to store the geometries and the id of each plot, it'll be used as a geometry filter and to calculate the % of area covered by the items.

In [5]:
samples_gdf = pd.read_pickle('shp/samples.p')

In [6]:
#create a geoDataFrame object from a .txt file 
if os.path.isfile(FILENAME):
    df = pd.read_csv(FILENAME, sep=' ')
    
    #filter only the `nb_rows` first rows
    nb_rows = 1000#len(df)
    filter_df  = df[df.index.isin(range(nb_rows))]
    df = filter_df
    
    #create the geodataframe 
    pts = [Point(df.loc[i][FILE_LNG], df.loc[i][FILE_LAT]) for i in range(len(df))]
    samples_gdf = gpd.GeoDataFrame(data={'geometry': pts}, index=df[FILE_ID], crs="EPSG:4326")
    samples_gdf.index.names = ['id']
    samples_gdf = samples_gdf.to_crs('ESRI:54032')
    samples_gdf['geometry'] = samples_gdf['geometry'].buffer(BUFFER_SIZE)
    samples_gdf = samples_gdf.to_crs("EPSG:4326")
    samples_gdf['geometry'] = samples_gdf['geometry'].envelope

In [7]:
samples_gdf = samples_gdf[400:500]

In [13]:
samples_gdf = samples_gdf[:10]

### Connect to client

In [8]:
client = api.ClientV1(api_key=PLANET_API_KEY)

### Define filters

In [9]:
# define test data for the filter
start_date = datetime.datetime(2009, 1, 1)
stop_date = datetime.datetime(2020, 12, 31)
cloud_cover_lte = 0.01
minimum_covered_area = 90 # included

### Define scores

In [10]:
# item_type_score
item_type_score = {
    'PSScene4Band':8, 
    'PSScene3Band':8, 
    'PSOrthoTile':10,
    'REOrthoTile':0,
    'SkySatScene':0,
}

# season score
months_score = {
    1: 5, 7:0,
    2: 5, 8:0,
    3: 5, 9:0,
    4: 0, 10:7,
    5: 0, 11:10,
    6: 0, 12:10,
}

# cloud_score

def cloud_score(cloud_cover):
    """ Define the cloud cover threshold and score
    
    1 = 1%
    
    """
    cloud_cover = cloud_cover*100
    
    if cloud_cover == 0:
        return 10
    elif cloud_cover <= 1 and cloud_cover > 0:
        return 5
    else:
        return 0
    
# Covered area

def cover_score(covered_area):
    """Define the cover area threshold and score
    """
    covered_area = covered_area*100
    
    if covered_area >= 99:
        return 10
    
    elif covered_area >= 95:
        return 5
    
    else:
        return 0
    


## OPTION: 1.1 Get items for individual samples ((optional))
### Get items and metadata using filters

In [ ]:
# Define AOI, by selecting the first row of the samples geodataframe
# For this example, we are going to use the first sample
row_number = 2
aoi_geometry = json.loads(dumps(samples_gdf.iloc[row_number].geometry))
sample_id = samples_gdf.iloc[row_number].name

In [ ]:
%%time
request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)
items = get_items(sample_id, request, client)

# Transform items into a pandas dataframe with useful columns
metadata_df = get_dataframe(items)

### Calculate percentage of covered area

Calculate the percentage of covered area from the sample area with the item footprint

In [ ]:
# Mutate metadata_df and add the percentage of cover area
add_cover_area(metadata_df, samples_gdf)

In [ ]:
# Remove items that are under 90% of covered area
metadata_df = metadata_df[metadata_df.cover_perc >= (minimum_covered_area/100)]

### Score items


In [ ]:
scored_items = score_items(metadata_df, item_type_score, months_score, cloud_score, cover_score)

In [ ]:
selected_items = get_one_item_per_year(scored_items)

### ((Optional)): Export thumbnails
Create thumbnails from the selected items (dataframe) and store them into a structured folder

In [ ]:
# save_thumb(selected_items)

## OPTION 1.2 Get items for all plots and store into a big df
### Loop over all plots
Loop over all plots and get the items.

In [45]:
# Create a list of dataframes 

LOG_PATH = os.path.join(os.getcwd(), 'logs')
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)

def run_multiprocess(index, row):    
    aoi_geometry = json.loads(dumps(row.geometry))
    sample_id = row.name
    
    now = datetime.now()
    formated_now = now.strftime('%Y%m%d_%H_%M')
    srch_log_file = os.path.join(LOG_PATH, f'search_logs_{formated_now}.txt')
    
    OUT_PIKL_PATH = os.path.join(os.getcwd(), 'searches', formated_now)
    Path(OUT_PIKL_PATH).mkdir(parents=True, exist_ok=True)

    
    pickle_df_name = os.path.join(OUT_PIKL_PATH, str(sample_id)+'.p')

    if not os.path.exists(pickle_df_name):
        request = build_request(aoi_geometry, start_date, stop_date, cloud_cover_lte)

        try:
            print(f'Starting {sample_id}')
            items = get_items(sample_id, request, client)
            # Transform items into a pandas dataframe with useful columns
            metadata_df = get_dataframe(items)
            
            # Mutate metadata_df and add the percentage of cover area
            add_cover_area(metadata_df, samples_gdf)

            # Remove items that are under the minimum_covered_area threshold
            metadata_df = metadata_df[metadata_df.cover_perc >= (minimum_covered_area/100)]

            # Create a score for each item
            scored_items = score_items(metadata_df, item_type_score, months_score, cloud_score, cover_score)

            # Filter scored_items and get only one per year
            selected_items = get_one_item_per_year(scored_items)
            
            # Save into a pickled file
            selected_items.to_pickle(pickle_df_name)
            
            print(f'{sample_id} pickled.')
            
        except Exception as e:
            with open(srch_log_file, 'a') as lf:
                
                lf.write(f'Sample {sample_id}:{e}\n')

    else:
        print(f'Search for {sample_id} already saved.')

In [47]:
%%time
if __name__ == '__main__':
    pool = multiprocessing.Pool(10)
    for index, row in samples_gdf.iterrows():
        pool.apply_async(run_multiprocess, args=(index, row,))
    pool.close()
    pool.join()

Starting 594399Starting 599669Starting 604840Starting 603303Starting 605677
Starting 604844Starting 606868
Starting 605725

Starting 602611

Starting 606917



594399 pickled.
605677 pickled.
603303 pickled.
604840 pickled.
605725 pickled.
606917 pickled.
604844 pickled.
602611 pickled.
599669 pickled.
606868 pickled.
CPU times: user 76 ms, sys: 47.5 ms, total: 124 ms
Wall time: 40.1 s


### Read all the pickled files, merge and store them in a big df

In [48]:
pkl_list = glob.glob(os.path.join(OUT_PIKL_PATH, '*p'))
all_df = pd.concat([pd.read_pickle(pkl) for pkl in pkl_list])

In [49]:
all_df

1098

# 2. Order assets
### Create json request

In [50]:
products_bundles = {

    # Is not possible to ask for analytic_dn in PSScene3Band, so the next option is visual
    # for more info go to https://developers.planet.com/docs/orders/product-bundles-reference/
    'PSScene3Band': "analytic,visual",
    'PSScene4Band': "analytic_udm2,analytic_sr,analytic",
    'PSOrthoTile': "analytic_5b_udm2,analytic_5b,analytic_udm2,analytic",
    'REOrthoTile': "analytic",
}

In [51]:
# To create the order we need a dataframe with filtered items,
# and a samples_gdf with sample_id and geometry to clip each item.

# Build an order for each sample and store in a orders_list
orders = []
samples_ids = list(all_df.sample_id.unique())
for idx, row in samples_gdf.iterrows():
    if idx in samples_ids:
        order = build_order_from_metadata(all_df, idx, row, products_bundles)
        orders.append(order)

### Request order
<font color='red'>The following lines will start the order in the planet server, once the order is placed and running, there is no way to stop it.</font>

NOTE: The following loop will skip the samples that have already been downloaded, however it's based on the existing_orders request, and we are not sure how long the requests will remain in the planet server.

In [64]:
# Request the existing orders and store their sample_id (name)
current_server_orders = get_existing_orders(client)
ordered_sample_names = [order['name'] for order in current_server_orders]
now = datetime.now()
formated_now = now.strftime('%Y%m%d_%H_%M')
ordr_log_file = os.path.join(LOG_PATH, f'order_logs_{formated_now}.txt')

orders_info = []
for new_order in orders:

    # Make sure that the sample is not already downloaded
    sample_name = new_order['name']
    if sample_name not in ordered_sample_names:
        
        try:
            # The following line will create the order in the server
            @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota, max_time=360)
            def place_order():
                return client.create_order(new_order).get()
            
            order_info = place_order()
            orders_info.append(order_info)
            
            order_id = order_info['id']
            sample_name = order_info['name']
            
            print(f'order {order_id} with {sample_name} has been placed.')
            
        except Exception as e:
            with open(ordr_log_file, 'a') as lf:
                
                lf.write(f'Sample {sample_name}:{e}\n')
    else:
        print(f'Skipping {sample_name}: already requested.')

Skipping sample_594399: already requested.
Skipping sample_599669: already requested.
Skipping sample_604840: already requested.
Skipping sample_606868: already requested.
Skipping sample_604844: already requested.
Skipping sample_605677: already requested.
Skipping sample_603303: already requested.
Skipping sample_606917: already requested.
Skipping sample_605725: already requested.
Skipping sample_602611: already requested.


## Get status
The following get_order_status line has to be re-runned everytime we want to know the orders statusw

In [84]:
# Use the pages to limit the search, every page will display 20 orders.
get_orders_status(client, pages=None).order_values()

created_on,last_message,last_modified,id,name,state


# 3. Download

In [ ]:
df = pd.DataFrame({'$a':[1,2], '$b': [10,20]})

In [ ]:
download_path = os.path.join(os.getcwd(),'downloads')

In [ ]:
LOG_PATH = os.path.join(os.getcwd(), 'logs')
Path(LOG_PATH).mkdir(parents=True, exist_ok=True)
dw_log_file = os.path.join(LOG_PATH, 'download_logs.txt')

In [ ]:
# Define the dates in which the desired oreders were ordered.

current_server_orders = get_existing_orders(client)

start_date = datetime.date(2020,9,10)
stop_date = datetime.date(2020,9,10)
success_states = ['success', 'partial']

for order in current_server_orders:
    
    created_on = pd.to_datetime(order['created_on']).date()
    state = order['state']
    
    if state in success_states:

        if created_on >= start_date and created_on <= stop_date:
            # Create the download folder
            download_order_path = os.path.join(download_path, order['name'])
            Path(download_order_path).mkdir(parents=True, exist_ok=True)

            # Check if folder is empty:
            if not os.listdir(download_order_path) :
                try:
                    print(f'downloading {order["name"]} ')
                    callback = api.write_to_file(directory=f'{download_order_path}/', overwrite=True)

                    @backoff.on_exception(backoff.expo,planet.api.exceptions.OverQuota,max_time=360)
                    def download():
                        client.download_order(order['id'], callback=callback)
                    download()
                    
                except Exception as e:
                    with open(dw_log_file, 'w') as lf:
                        lf.write('\n'.join(f'{sample_id}:{e}'))
            else:
                print(f'The folder {download_order_path} is not empty ')